# ML Project: Creating the Dataset

*Note: All datasets were updated in late October, early November.*

*At first, when updating, noticeable changes occurred. Instance count jumped from 50 to 60 to 75 within two weeks. Then, after about a month (early November), changes were insignificant and the number of countries reached its steady state with about 80 countries.*

We'll join multiple datasets on a common feature. The common feature is 'Country'

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 

In [2]:
cases_data=pd.read_csv("COVID dataset/covid_latest.csv")
population_data=pd.read_csv("COVID dataset/population.csv")
countries_data=pd.read_csv("COVID dataset/countries of the world.csv")
usefulfeatures_data=pd.read_csv("COVID dataset/countries useful features.csv")
GDP_data=pd.read_csv("COVID dataset/GDP.csv")
HF_data=pd.read_csv("COVID dataset/hf score.csv")
literacy1_data=pd.read_csv("COVID dataset/literacy rate 1.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'COVID dataset/covid_latest.csv'

In [ ]:
cases_data.info()

Select the desired feature from the datasets then join them. We chose population and density.

Make sure that the column 'Country' is written identicaly 

In [ ]:
population_data.info()

In [ ]:
population_data=population_data.rename(columns={'Country (or dependency)':'Country','Density (P/Km²)':'Density','Population (2020)':'Population'})

population_data=population_data.filter(items=['Country','Population','Density'])

cases_data=cases_data.rename(columns={'location':'Country'})

cases_data=cases_data.filter(items=['date','Country','total_cases','total_tests','tests_per_case','extreme_poverty','human_development_index','hospital_beds_per_thousand','handwashing_facilities','continent','reproduction_rate','total_deaths','positive_rate'])

result=population_data.join(cases_data.set_index('Country'), on='Country')

Change 'date' from object to datetime. datetime allows us to apply helpful operations of our date features

In [ ]:
result['date'] = result['date'].apply(pd.to_datetime)

Only select countries where cases reached at least 1% of its population. 

Apply minimum function. Now the date when cases reached 1% is selected.

In [ ]:
result.info()

In [ ]:
#mortality_rate=result.loc[result['date']=="2021-02-08"]['total_deaths']/result.loc[result['date']=="2021-02-08"]['total_cases']

In [ ]:
#result['mortality_rate']=mortality_rate
#result['reproduction_rate_mortality'] = result.loc[result['date']=="2021-01-22"]['reproduction_rate']
#result['total_deaths_mortality'] = result.loc[result['date']=="2021-02-05"]['total_deaths']

In [ ]:
#result[['Country','mortality_rate']]

In [ ]:
result2=result.loc[result['total_cases']>=0.05*result['Population']]

result2=result2.set_index('Country')

result2=result2.min(axis=0,level='Country')

In [ ]:
result2['mortality_rate'] = result2['total_deaths']/result2['total_cases']

In [ ]:
result2.info()

Join with the next dataset (countries_data). We'll select 'Climate' from this dataset.

Note: We applied str.strip() to remove spaces from the begining and end of the country column. This dataset (countries of the world) has a space at the begining of each country. To correctly join them the 'Country' column has to be identical. 

In [ ]:
countries_data.info()

In [ ]:
result2=result2.reset_index()

countries_data=countries_data.filter(items=['Country','Region','Climate'])
countries_data['Country']=countries_data['Country'].str.strip()

We noticed that we don't need the feature 'Region' as we'll be using 'continent' from another dataset.

Drop features with NULL climate.

In [ ]:
countries_data=countries_data.dropna(subset=["Climate"])
countries_data=countries_data.reset_index(drop=True)

countries_data.drop('Region',axis='columns', inplace=True)

countriesData=result2.set_index('Country').join(countries_data.set_index('Country'))

Join with the next dataset (useful features data). 

We'll select Tourism, Date of first case, Latitude, Longitude, Lockdown date, and Lockdown type.

In [ ]:
usefulfeatures_data.info()

In [ ]:
countriesData=countriesData.reset_index()

usefulfeatures_data=usefulfeatures_data.filter(items=['Country_Region','Tourism','Date_FirstConfirmedCase','Latitude','Longtitude','Lockdown_Date','Lockdown_Type'])
usefulfeatures_data.info()

Change date from object to datetime

In [ ]:
usefulfeatures_data=usefulfeatures_data.rename(columns={'Country_Region':'Country'})

usefulfeatures_data['Lockdown_Date']=usefulfeatures_data['Lockdown_Date'].apply(pd.to_datetime)
usefulfeatures_data['Date_FirstConfirmedCase']=usefulfeatures_data['Date_FirstConfirmedCase'].apply(pd.to_datetime)

countriesData=countriesData.set_index('Country').join(usefulfeatures_data.set_index('Country'))

Rename the date column. Drop total cases, we don't need it.

In [ ]:
countriesData=countriesData.reset_index()

countriesData=countriesData.rename(columns={'date':'Date to reach 1%'})

#countriesData=countriesData.drop(['total_cases'], axis=1)

Join the next dataset (GDP). We'll use the feature GDP from this dataset.

In [ ]:
GDP_data.info()

In [ ]:
GDP_data=GDP_data.filter(items=['Country ','2017','2018'])

GDP_data=GDP_data.rename(columns={'Country ':'Country'})

From the GDP dataset, we'll be using values collected from 2018. Some 2018 values are null so fill them with older 2017 GDP values.

Then, drop 2017 GDP column as we don't need it anymore

In [ ]:
GDP_data['2018']=GDP_data['2018'].fillna(GDP_data['2017'])

GDP_data=GDP_data.drop(['2017'], axis=1)

GDP_data=GDP_data.rename(columns={'2018':'GDP 2018'})

countriesData=countriesData.set_index('Country').join(GDP_data.set_index('Country'))

Join the next dataset (HF data). We'll select the human freedom score from this dataset.

In [ ]:
HF_data.info()

In [ ]:
HF_data=HF_data.filter(items=['countries','hf_score'])

HF_data=HF_data.rename(columns={'countries':'Country','hf_score':'HF score 2017'})

countriesData=countriesData.reset_index()

countriesData=countriesData.set_index('Country').join(HF_data.set_index('Country'))

In [ ]:
countriesData=countriesData.reset_index()
countriesData=countriesData.reset_index()

Join the next dataset (literacy data). We selected the average literacy rate.

In [ ]:
literacy1_data.info()

In [ ]:
literacy1_data=literacy1_data.filter(items=['country',' literacy_rate_percent_all'])

literacy1_data=literacy1_data.rename(columns={'country':'Country',' literacy_rate_percent_all':'literacy rate'})

countriesData=countriesData.set_index('Country').join(literacy1_data.set_index('Country'), on='Country')

countriesData=countriesData.reset_index()

We now have our dataset ready! 

Save it in the project repository.

In [ ]:
countriesData.to_csv('Full data333.csv',index=False)

In [ ]:
countriesData.info()

In [ ]:
countriesData.loc[countriesData['Country']=='Libya']

In [ ]:
plt.hist(countriesData['mortality_rate'])